# The problem

There are 100 fresh plums on a plum tree including 3 rotten ones. Every day, each plum has a 35% chance of rotting, if they are over 2 days old. Every day, 3 fresh plums grow. The farmer comes and takes 75% of the rotten plums every 4 days, stopping the rot by 2 days. After 2 weeks:
1. How many fresh plums are there?
2. How many rotten plums are still on the tree?
3. How many total plums are there?
4. How many rotten plums did the farmer collect?
5. How many rotten plums were there in total?

# The representations

## Observable Variables


Symbol | Notes
:---:|:------
$D$ | Day
$P$ | Total plums
$R$ | Number of rotten plums
$G$ | Number of good plums



## Hidden Variables and Parameters

Symbol | Notes
:---:|:------
$M$ | Number of plums subjected to the rotting diseases, i.e. good plums > 2 days
$N$ | Temporary variable ($N=R + 0.35 * M * f_1$)
$f_1$ | Farmer factor-1,to activate the rotten procedure, $f_1 =
\left\{
    \begin{array}{lll}
        0  & \quad \mbox{if within} &\mbox{2 days of farmer's picking} \\
        1  & \quad \mbox{if on and after}  &\mbox{2 days of farmer's picking} 
    \end{array}
\right.$
$f_2 $ | Farmer factor-2, to remove rotten plums, $f_2 =
\left\{
    \begin{array}{ll}
        0.75  & \quad \mbox{on Farmer's day} \\
        0     & \quad \mbox{else where} 
    \end{array}
\right.$
$f_3$ | For initial 2 days, $f_3 = 
\left \{
    \begin{array}{ll}
        0 & \quad \mbox{on Day 0, 1, 2}\\
        1 & \quad \mbox{on Day 3, onwards}
    \end{array}
\right.$


# Equations

Let $k$ be the day index
\begin{align}
P^{k+1} & = P^k + 3 - f_2 * N \\
R^{k+1} & = (1 -f_2) * N \\
G^{k+1} & = P^k + 3 - N \\
-- &- -------- \\
M & = f_3* (G^k -6) \\
N & = R^k + 0.35 * M * f_1\\
\end{align}
    

# Solve the equations

From here, there are two approaches to solve those equations -- brutal force, or NN (which offers something intelligent)

## Brutal force

We treat $M$ and $N$ as $M^{k+1}$ and $N^{k+1}$ for the $k+1$ day

\begin{align}
P^{k+1} + f_2 * N^{k+1} &= P^k + 3 \\
R^{k+1} - (1-f_2)*N^{k+1} &= 0 \\
G^{k+1} + N^{k+1} &= P^k + 3 \\
------- &- --- \\
M^{k+1} &= f_3 * G^k - 6* f_3 \\
-0.35 * f_1 * M^{k+1} + N^{k+1} &= R^k \\
\end{align}

Let $\mathbf{X}$ be 
$$\mathbf{X} = \begin{bmatrix}
P\\
R \\
G \\
--\\
M \\
N \\
\end{bmatrix}
$$


\begin{equation}
\begin{bmatrix} 
1 & 0 & 0 & 0 & f_2 \\
0 & 1 & 0 & 0 & -(1-f_2)\\
0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & -0.35 * f_1 & 1
\end{bmatrix} 
\begin{bmatrix}
P\\
R \\
G \\
M \\
N
\end{bmatrix}^{k+1} = 
\begin{bmatrix} 
1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 \\
0 & 0 & f_3 & 0 & 0 \\
0 & 1 & 0 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
P\\
R \\
G \\
M \\
N
\end{bmatrix}^k
+
\left[
\begin{array}{r}
3\\
0 \\
3 \\
-6*f_3 \\
0
\end{array}
\right]
\end{equation}

We then have 
\begin{equation}
W_1 \mathbf{X}^{k+1} = W_2 \mathbf{X}^k + B
\end{equation}

where
\begin{equation}
\begin{array}{}
W_1 = \begin{bmatrix} 
1 & 0 & 0 & 0 & f_2 \\
0 & 1 & 0 & 0 & -(1-f_2) \\
0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & -0.35*f_1 & 1
\end{bmatrix} 
,&
W_2 = \begin{bmatrix} 
1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 \\
0 & 0 & f_3 & 0 & 0 \\
0 & 1 & 0 & 0 & 0
\end{bmatrix},
&
B = \left[
\begin{array}{r}
3 \\
0 \\
3 \\
-6*f_3 \\
0
\end{array}
\right]
\end{array}
\end{equation}

In [75]:
import numpy as np
import scipy as sp


## Define a function to do the calculation

To calculate today's Y, based on parameters f1 and f2, and the values from the previous day.

In [76]:
def dx_cal(X, f1, f2, f3) :
    W1 = np.array([[1,  0, 0, 0, f2],
                   [0,  1, 0, 0, -(1-f2)],
                   [0,  0, 1, 0, 1],
                   [0,  0, 0, 1, 0],
                   [0,  0, 0, -0.35*f1, 1]])

    W2 = np.array([[1, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 0, f3, 0, 0],
                   [0, 1, 0, 0, 0]])

    B = np.array([[ 3, 0, 3,-6*f3, 0]]).T
    
    W1_inv = np.linalg.inv(W1)
    
    Y = W1_inv.dot(W2.dot(X) + B)
    return np.rint(Y)


In [81]:
D0 = np.array([[100, 3, 97, 0, 0]]).T

In [82]:
D1 = dx_cal(D0, 1, 0, 0)

In [83]:
D1.T

array([[ 103.,    3.,  100.,    0.,    3.]])

In [84]:
D2 = dx_cal(D1, 1, 0, 0)

In [85]:
np.rint(D2.T[0])

array([ 106.,    3.,  103.,    0.,    3.])

In [86]:
D3 = dx_cal(D2, 1, 0, 1)

In [87]:
np.rint(D3.T[0])

array([ 109.,   37.,   72.,   97.,   37.])

In [88]:
D4 = dx_cal(D3, 1, 0.75, 1)

In [91]:
D4.T[0]

array([ 67.,  15.,  52.,  66.,  60.])

In [92]:
D5 = dx_cal(D4, 0, 0, 1)

In [93]:
D5.T[0]

array([ 70.,  15.,  55.,  46.,  15.])

In [94]:
D6 = dx_cal(D5, 0, 0, 1)

In [95]:
D6.T[0]

array([ 73.,  15.,  58.,  49.,  15.])